# LIBRARY IMPORTS

In [67]:
%run importing.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# LOAD DATA

In [68]:
df = pd.read_csv('train.csv')

# EDA

In [69]:
X, y = eda(df)

# PREPROCESSING

In [70]:
pX, py = process(X, y)

# SPLIT DATA

In [71]:
split_size = 0.25
X_train, X_val, y_train, y_val = train_test_split(pX, py, test_size=split_size, random_state=42)
X_train.shape, X_val.shape

((1095, 265), (365, 265))

# MODELING

In [38]:
lin = LinearRegression().fit(X_train, y_train)
knn = KNeighborsRegressor().fit(X_train, y_train)
rfr = RandomForestRegressor().fit(X_train, y_train)
abr = AdaBoostRegressor().fit(X_train, y_train)
gbr = GradientBoostingRegressor(n_estimators=400, max_depth=3).fit(X_train, y_train)
cbr = CatBoostRegressor(verbose=False, iterations=2000).fit(X_train, y_train)

In [39]:
print('lin:', mean_squared_error(y_val, lin.predict(X_val)) ** 0.5)
print('knn:', mean_squared_error(y_val, knn.predict(X_val)) ** 0.5)
print('rfr:', mean_squared_error(y_val, rfr.predict(X_val)) ** 0.5)
print('abr:', mean_squared_error(y_val, abr.predict(X_val)) ** 0.5)
print('gbr:', mean_squared_error(y_val, gbr.predict(X_val)) ** 0.5)
print('cbr:', mean_squared_error(y_val, cbr.predict(X_val)) ** 0.5)

lin: 562858366395.3821
knn: 0.205709026563613
rfr: 0.14738060502461142
abr: 0.17692363752623116
gbr: 0.1310708612533411
cbr: 0.19864638307380428


In [83]:
model = Sequential()
model.add(Dense(units=2096, activation='relu', input_shape=(265,)))
model.add(Dropout(0.25))
model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=1, activation='linear'))
opt = optimizers.SGD()
model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_33 (Dense)             (None, 2096)              557536    
_________________________________________________________________
dropout_22 (Dropout)         (None, 2096)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 1024)              2147328   
_________________________________________________________________
dropout_23 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 1025      
Total params: 2,705,889
Trainable params: 2,705,889
Non-trainable params: 0
_________________________________________________________________


In [84]:
history = model.fit(X_train, y_train, epochs=201, batch_size=25, verbose=0, validation_split=0.3,
                    shuffle=True, callbacks=[Logger(50)])

epoch: 0 {'loss': 27.833122265868024, 'rmse': 4.1639366, 'val_loss': 8.045459983559002, 'val_rmse': 2.4842017}
epoch: 50 {'loss': 0.463943091875268, 'rmse': 0.5382911, 'val_loss': 1.5028344505101352, 'val_rmse': 0.64541644}
epoch: 100 {'loss': 0.3624238377685024, 'rmse': 0.47482777, 'val_loss': 1.4668998012607946, 'val_rmse': 0.70271987}
epoch: 150 {'loss': 0.32447827566541515, 'rmse': 0.4504358, 'val_loss': 1.4024377434449355, 'val_rmse': 0.6609797}
epoch: 200 {'loss': 0.30408452313111284, 'rmse': 0.44305304, 'val_loss': 1.3936251133770927, 'val_rmse': 0.6839174}


In [35]:
model = GradientBoostingRegressor(n_estimators=400, max_depth=3).fit(pX, py)

In [36]:
df_kaggle = pd.read_csv('test.csv')
X_kaggle, _ = eda(df_kaggle, False)
X_kaggle, _ = process(X_kaggle)

In [37]:
dict_kaggle = {'Id' : df_kaggle['Id'], 'SalePrice': np.exp(model.predict(X_kaggle))}
pd.DataFrame(dict_kaggle).to_csv('predictions.csv', index=False)